In [1]:
from IPython.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

%load_ext autoreload
%autoreload 2

import warnings
warnings.filterwarnings("ignore")

In [2]:
import pandas as pd
from factor_data import FactorData
from price_data import PriceData
from factor_model import FactorModel
from pca import PCA
from clustering import Cluster
from find_pairs import PairsTrade
from allocator import Allocator
from pma import PMA


from matplotlib import style
import matplotlib.pyplot as plt
plt.style.use('bmh')

from datetime import datetime
from dateutil.relativedelta import relativedelta

In [3]:
%%time

def generate_datelist(start_date, end_date, freq="w"):
    if freq=="w":
        date_list = pd.date_range(start=start_date, end=end_date, 
                             freq='W-MON').strftime('%Y-%m-%d').tolist()
        
    else:
        date_list =[str(d)+"-01" for d in list(pd.period_range(start=start_date, end=end_date, freq='M'))]
    
    return date_list



date_list = generate_datelist("2020-12-01","2022-12-01",freq="m")
pma = PMA(1000000,date_list[0],allocation_per_trade=50000) # create a pma to record trades

sample_length = 3 # the length of the sample period
display = False # to show the pairs trade graphs


for da in date_list:

    
    print(F"Running ..... {da}")
    
    dt = datetime.strptime(da, '%Y-%m-%d')
    start = str(dt - relativedelta(years=sample_length))[:10] # get start from for the sample period data
    
    lookback_start = str(dt - relativedelta(months=1))[:10]
    
    # get price data for the sample period
    sample_data = PriceData(["all"], start=start, end=da, ticker_ref_date=da, refresh=False)
    
    # get price data for the out of sample period
    lookback_data = PriceData(["all"], start=lookback_start, end=da, ticker_ref_date=da, refresh=False)
    
    # get the factor data for the sample period
    factors = FactorData("all", frequency="m", start=start, end=da)
    
    # run the ts regression to get factor betas
    _factor_model = FactorModel(
                                sample_data.asset_return, 
                                factors.factor_data,
                                ts_pvalues_threshold=0.05,
                                factor_hard_cap=15,
                                coff_std_limit=3
    )

    # apply PCA to the data
    _pca = PCA(_factor_model.clustering_coefficients,cum_explain_threshold=0.80)

    # clsuter the data
    cluster = Cluster(
                      _pca.pca,
                      method="GMM",
                      number_of_cluster=None,
                      cluster_cap=25,
                      cluster_floor=2,
                      graph=display
                    )

    # find pairs
    pairs_trade = PairsTrade(
                             _factor_model,
                             cluster, 
                             sample_data, 
                             lookback_data,
                             allocation_date=da,
                             entry_zscore=3,
                             stop_loss_zscore=5,
                             test_percentile=0.5,
                             stationary_pvalue=0.05,
                             display=display
                            )


    
    # put the trades into the allocator
    allocator = Allocator(pairs_trade.pairs,trade_date=da)
    
    # add the trades into the pma
    pma.add_to_pma(allocator.orders,allocator.info,da)
    
    
    # run pma to calculate aum
    pma.run_pma(da)
    
    
    print("""
    
---------------------------------------------------------------

""")
    

Running ..... 2020-12-01
    289 of ticker.
--------- Runing Cluster 0 ---------------
--------- Runing Cluster 1 ---------------
--------- Runing Cluster 2 ---------------
--------- Runing Cluster 3 ---------------
--------- Runing Cluster 4 ---------------
--------- Runing Cluster 5 ---------------
--------- Runing Cluster 6 ---------------
Date: 2020-12-01 -- AUM: 1000000.0

    
---------------------------------------------------------------


Running ..... 2021-01-01
    280 of ticker.
--------- Runing Cluster 0 ---------------
--------- Runing Cluster 1 ---------------
--------- Runing Cluster 2 ---------------
Date: 2021-01-01 -- AUM: 1002551.09

    
---------------------------------------------------------------


Running ..... 2021-02-01
    317 of ticker.
--------- Runing Cluster 0 ---------------
--------- Runing Cluster 1 ---------------
--------- Runing Cluster 2 ---------------
--------- Runing Cluster 3 ---------------
--------- Runing Cluster 4 ---------------
Sent Tra

    460 of ticker.
--------- Runing Cluster 0 ---------------
--------- Runing Cluster 1 ---------------
--------- Runing Cluster 2 ---------------
--------- Runing Cluster 3 ---------------
--------- Runing Cluster 4 ---------------
Sent Trade to Exchange....

        
                  Closed 8 Trades, Number of Allocation Reduced from 8 to 0
                  Cash Amount went from 1298764.59 to 1027864.02
                  With Exchange Fee of 1505.28

                  
Sent Trade to Exchange....

        
                  Added 6 Trades, Number of Allocation Increased from 0 to 6
                  Cash Amount went from 1027864.02 to 1020354.96
                  With Exchange Fee of 1497.6
                  
Date: 2021-12-01 -- AUM: 1035069.9

    
---------------------------------------------------------------


Running ..... 2022-01-01
    444 of ticker.
--------- Runing Cluster 0 ---------------
--------- Runing Cluster 1 ---------------
--------- Runing Cluster 2 -------------

--------- Runing Cluster 3 ---------------
--------- Runing Cluster 4 ---------------
--------- Runing Cluster 5 ---------------
--------- Runing Cluster 6 ---------------
Sent Trade to Exchange....

        
                  Closed 2 Trades, Number of Allocation Reduced from 2 to 0
                  Cash Amount went from 1025167.65 to 1005895.03
                  With Exchange Fee of 429.45

                  
Sent Trade to Exchange....

        
                  Added 4 Trades, Number of Allocation Increased from 0 to 4
                  Cash Amount went from 1005895.03 to 910137.83
                  With Exchange Fee of 1001.92
                  
Date: 2022-10-01 -- AUM: 1020888.07

    
---------------------------------------------------------------


Running ..... 2022-11-01
    323 of ticker.
--------- Runing Cluster 0 ---------------
--------- Runing Cluster 1 ---------------
--------- Runing Cluster 2 ---------------
--------- Runing Cluster 3 ---------------
--------- Runing

In [4]:
pma.aum_history.to_csv("backtest/GMM-1-8-historical_aum.csv")
pma.number_of_trade.to_csv("backtest/GMM-1-8-number_of_trade.csv")
pma.total_long_position.to_csv("backtest/GMM-1-8-long.csv")
pma.total_short_position.to_csv("backtest/GMM-1-8-short.csv")